In [1]:
### SLICER
## setup mean model
## align model center to CT center
## optionally invert axes
## scale model to 0.5*CT_ROI
## define manual landmarks on CT
## optimize to match manual landmarks


### GREY LEVEL MODEL
## align SSM sample to CT-segmented-model
## define landmarks
## extract profiles
## compute covariance matrix

# DEBUG

In [203]:
from extract_profile import *
%matplotlib

patientName = "V2"

def convertRasToIjk(points):
    points = np.c_[points, np.ones(points.shape[0])]
    pointsIjk = np.einsum('ab,ib->ia', RASToIjk, points)[:,:-1]
    return pointsIjk

inputVolumePath = inputVolumePathTemplate.format(patientName)
inputVolumeNib = nib.load(inputVolumePath)
print("Loading nifti...", end=" ")
inputVolumeNumpy = inputVolumeNib.get_fdata()
print("done!")
ijkToRASMatrix = inputVolumeNib.affine
RASToIjk = np.linalg.inv(ijkToRASMatrix)

# define interpolation method for the image volume
ii, jj, kk = [np.arange(0,inputVolumeNumpy.shape[i]) for i in range(3)]
interpolator = scipy.interpolate.RegularGridInterpolator(
    (ii, jj, kk),
    inputVolumeNumpy,
    method='linear',
    bounds_error=False,
    fill_value=0.,
)
SSMModelPath = SSMModelPathTemplate.format(patientName)
SSMModel = pv.read(SSMModelPath)
LPSToRASMatrix = np.diag([-1,-1,1,1])
if convertSSMModelsToRAS:
    SSMModel = SSMModel.transform(LPSToRASMatrix)
origModelPath = origModelPathTemplate.format(patientName)
origModel = pv.read(origModelPath)

if convertSSMModelsToRAS:
    origModel = origModel.transform(LPSToRASMatrix)
SSMTransformed = ICP(SSMModel, origModel)
SSMTransformed = SSMTransformed.compute_normals(cell_normals=False)

#SSMTransformed.save('SSMTrasformed.vtp')

# # define sample points
# distances = scipy.spatial.distance.pdist(SSMTransformed.points , metric='euclidean')
# distances = scipy.spatial.distance.squareform(distances, force='tomatrix', checks=True)
# nSamplePoints = 500
# samplePointsIdxs = getGreedyPerm(distances, nSamplePoints)[0]
# samplePoints = SSMTransformed.points[samplePointsIdxs]
# np.savetxt('samplePoints.txt', samplePointsIdxs, fmt="%i", delimiter=',')
# pv.PolyData(samplePoints).save('samplePointsForA2Model.vtp')

samplePointsIdxs = np.loadtxt('samplePoints.txt', dtype=int)
samplePoints = SSMTransformed.points[samplePointsIdxs]
sampleNormals = SSMTransformed['Normals'][samplePointsIdxs]

def computeProfilePoints(point, direction):
    t = np.arange(-profileLen/2., (profileLen)/2 + profileSpacing, profileSpacing)
    # (t.shape[0], nSamplePoints, 3)
    return point + np.einsum('i,kj->ikj', t, direction)

# compute coordinates of sampling points (near the model's surface)
profilePoints = computeProfilePoints(samplePoints, sampleNormals)
# convert sampling points coordinate to IJK space to sample the image
pointsIjk = convertRasToIjk(profilePoints.reshape(-1,3))
profiles = interpolator(pointsIjk).reshape(profilePoints.shape[:-1])

profiles_norm_grad = np.gradient(profiles, profileSpacing, axis=0)
profiles_norm_grad = profiles_norm_grad / profiles_norm_grad.std(axis=0)

if not osp.isdir(profilesDir):
    os.makedirs(profilesDir)
profilePathTemplate = osp.join(profilesDir, "{}.txt")
np.savetxt(profilePathTemplate.format(patientName), profiles_norm_grad.T.reshape(-1), fmt='%.5f')

# plots
profilesPlotsDir = osp.join(profilesDir, "plots")
if not osp.isdir(profilesPlotsDir):
    os.makedirs(profilesPlotsDir)
plotsPathTemplate = osp.join(profilesPlotsDir, "{}.png")
if plots:
    t = np.arange(-profileLen/2., (profileLen)/2 + profileSpacing, profileSpacing)
    plt.errorbar(t, profiles_norm_grad.mean(-1), yerr=profiles_norm_grad.std(-1))
    plt.xlabel("displacement from surface [mm]")
    plt.ylabel("normalized gradient")
    plt.title("{} profiles mean".format(patientName))
    #plt.savefig(plotsPathTemplate.format(patientName), dpi=80)
    #plt.close()
    plt.show()

In [213]:
origModel.save('orig.vtp')
SSMTransformed.save('transformed.vtp')
#pv.PolyData(convertRasToIjk(SSMTransformed.points)).save('SSMTransformed_IJKSpace.vtp')

# covariance matrix

In [184]:
import numpy as np
import glob
import sklearn.covariance as Covariance

profilesPaths = glob.glob("./profiles/*.txt")

profiles = []
for p in profilesPaths:
    profiles.append(np.loadtxt(p))

profiles = np.stack(profiles)[:,:]

p = profiles[:,:]# - profiles[:,:500].mean(0)
#rank of covariance matrix
#(scipy.linalg.svdvals(np.dot(p.T, p)) > 1e-8).sum()

def get_covariance_object(X, load=True):
    if load:
        covarianceDict = np.load('./profiles/covarianceDict.npy', allow_pickle=True)[()]
        cov_object, mean = covarianceDict['cov_object', 'mean']
        return cov_object, mean
    
    mean = X.mean(0)
    cov_object = Covariance.OAS(assume_centered=True).fit(X-mean)
    #cov_object = Covariance.EmpiricalCovariance(assume_centered=True).fit(X-mean)
    #cov_object = Covariance.ShrunkCovariance(assume_centered=True, shrinkage=0.01).fit(X-mean)
    #cov_object = MinCovDet(assume_centered=True).fit(X-mean)
    #cov_object = Covariance.GraphicalLassoCV(assume_centered=True).fit(X-mean)
    covarianceDict = {
        "cov_object" : cov_object,
        "mean" : mean,
    }
    np.save('./profiles/covarianceDict.npy', covarianceDict)
    
    #i = 300
    #G=10
    #plt.title("covariance matrix")
    #plt.imshow(oas.covariance_[21*i:21*(i+G),21*i:21*(i+G)])
    #plt.colorbar()
    #plt.show()
    #
    #i = 400
    #G=5
    #plt.plot(profiles[0][21*i:21*(i+G)], label="sample")
    #plt.plot(profiles.mean(0)[21*i:21*(i+G)], label="average")
    #plt.legend()
    #plt.show()
    return cov_object, mean
    
def mahalanobis(cov_object, X, mean):
    return cov_object.mahalanobis(X-mean)